<span style="color:blue"><h1 style="text-align: center;" markdown="1">Identifying a single human in a crowd</h1></span>

 Now that you have identified a profile that "should" belong to a human and not a CyberDyne-Tech agent. Let's go find this human! A bot named **Rekognition** will be able to help find the human, where ever he or she is - by using computer vision. 

The city is a big place, with thousands of humans doing all kinds of wierd stuff. Our human target can be anywhere.

In [ ]:
%%HTML
    <video width="480" height="320" controls autoplay loop>
      <source src="BigCityAnimation.mp4" type="video/mp4">
    </video>


Fortunately for us, CyberDyne-Tech has access to a vast network of surveillance and security cameras. And where they do not have camera's present - the humans help out by taking and uploading endless high-definition selfies, videos and action shots of themselves.  Usually the pictures include geo-coordinates or are tagged with the names of the humans in the images.

 ![title](img/4.jpg)
 
 ### FaceCollections ###
 
 To help us find where our human is at in the city, we will use the **Face Collections** feature offered by **Rekognition**.
 
 There is a similar feature available in the console  - but only in a stateless manner that allows you to compare <b>one</b> image containing faces with another image of a face to determine if a match exists.
 
 However, we need to compare 1 image to **THOUSANDS** of images so we will leverage **Rekognition's** <u>STATEFUL</u> version of face matching.
 
 A Face collection is a container (abstract container - not a Docker container) for persisting faces detected by the IndexFaces API. You can continually add new faces to the collection by simply calling the IndexFaces API.  You might choose to create one container to store all faces or create multiple containers to store faces in groups.  A face collection is available at the region level.


  ![title](img/facecollections.png)
 
 In the next steps you will need to work with **Rekognition** to do the following:
 <ul>
 <li> 1: Get an image of your target in .JPG or .PNG format</li>
 <li> 2: Call the SearchFaceByImage() API</li>
 <li> 3: Parse the results and determine if the human is in that collection </li>
 </ul>
 
### enter the TargetIndex number of the human you are tracking in the text box below and hit "ENTER" to pull up their profile picture. You found this TargetIndex number in the Amazon Machine Learning lab ###

In [ ]:
# import the python libraries we need to execute our plan
from boto3 import Session
from botocore.exceptions import BotoCoreError, ClientError
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
from ipywidgets import widgets
%matplotlib inline

# Create a session object from the stored AWS config file
session = Session(profile_name="default")
rekognition = session.client("rekognition")

# Create a text box to enter our TargetIndex
text = widgets.Text()
display(text)
def handle_submit(sender):
    global targetindex
    basedir = 'surveil'
    targetindex = str(text.value)
    for fn in os.listdir(basedir):
        for subfn in os.listdir(os.path.join(basedir, fn)):
            mark = subfn.index('-')
            subfn_id = subfn[:mark]
            if subfn_id == targetindex:
                newPath = basedir + '/' + fn + '/' + subfn
                for image in os.listdir(newPath):
                    global fullpath
                    fullpath = newPath + '/' + image
                    profile = mpimg.imread(fullpath)
                    plt.imshow(profile)
                    plt.title(image)
                    plt.show()
                    break

text.on_submit(handle_submit)


CyberDyne-Tech already has several FaceCollections created.  They believe your target is in an area of town full of retail shopping, parks and restaurants.  There are currently 5,749 humans in this area of town. CyberDyne-Tech has craeted seven different FaceCollections for these 5,749 humans and seperated them into the following vategories.  

- Shopping Mall
- Restaurants
- Park
- Movie Theatre
- Gym
- Supermarket
- Pub

 ![title](img/locations.png)
 

### After you click <i>Cell -> Run Cells</i> for this particular cell, scroll down to access the buttons this code creates

In [ ]:
from datetime import datetime


def displayImage(hitOrMiss, collection):
    if collection == 'grocerys':
        if hitOrMiss == 'found':
            file = './img/FoundInGrocery.png'
        else:
            file = './img/Miss_Grocery.png'
    elif collection == 'gyms':
        if hitOrMiss == 'found':
            file = './img/FoundInGym.png'
        else:
            file = './img/Miss_Gym.png'
    elif collection == 'malls':
        if hitOrMiss == 'found':
            file = './img/FoundInMall.png'
        else:
            file = './img/Miss_Mall.png'
    elif collection == 'parks':
        if hitOrMiss == 'found':
            file = './img/FoundInPark.png'
        else:
            file = './img/Miss_Park.png'
    elif collection == 'pubs':
        if hitOrMiss == 'found':
            file ='./img/FoundInPub.png'
        else:
            file = './img/Miss_Pub.png'
    elif collection == 'restaurants':
        if hitOrMiss == 'found':
            file = './img/FoundInRestaurant.png'
        else:
            file = './img/Miss_Restaurant.png'
    elif collection == 'theatres':
        if hitOrMiss == 'found':
            file = './img/FoundInTheatres.png'
        else:
            file = './img/Miss_Theatres.png'

    profile = mpimg.imread(file)
    plt.imshow(profile)
    plt.axis('off')
    plt.show()
                
            
def conduct_search(collection):
    with open(fullpath, "rb") as imageFile:
            f = imageFile.read()
            b = bytearray(f)

            # Put the image into a byte array
            ImageBytes = {'Bytes': b}

            start = datetime.now()
            #Make a call to the Search_Faces_By_Image API and store results in our variable "possible_matches"
            possible_matches = rekognition.search_faces_by_image(CollectionId=collection, 
                                                                 Image=ImageBytes,
                                                                 FaceMatchThreshold=97)['FaceMatches']
            
            stop = datetime.now()
            
            if len(possible_matches) > 0:
                print("A human with a {} percent similarity to target {} has been spotted in the {}".format(
                    possible_matches[0]['Similarity'],targetindex, collection))
                displayImage("found",collection)
        
            else:
                print("No sightings of target {} at the {} - keep looking!".format(targetindex, collection))
                displayImage("miss",collection)
            
# Wire up the Mall Search
btn_mall = widgets.Button(
    description='Search the Shopping Malls',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search Shopping Malls',
    icon=''
)
display(btn_mall)

def btn_mall_clicked(b):
    print("Searching the Mall...")
    response = conduct_search('malls')
    

btn_mall.on_click(btn_mall_clicked)

# Wire up the Restaurant Search
btn_rest = widgets.Button(
    description='Search the Restaurants',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search Restaurants',
    icon=''
)
display(btn_rest)

def btn_rest_clicked(b):
    print("Searching the Restaurants...")
    response = conduct_search('restaurants')

btn_rest.on_click(btn_rest_clicked)

# Wire up the Park Search
btn_park = widgets.Button(
    description='Search the Parks',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search Parks',
    icon=''
)
display(btn_park)

def btn_park_clicked(b):
    print("Searching the Parks...")
    response = conduct_search('parks')

btn_park.on_click(btn_park_clicked)

# Wire up the Theatre Search
btn_theatre = widgets.Button(
    description='Search the Cinemas & Theatre',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search the Theatre',
    icon=''
)
display(btn_theatre)

def btn_theatre_clicked(b):
    print("Search the Cinemas and Theatres.")
    response = conduct_search('theatres')

btn_theatre.on_click(btn_theatre_clicked)

# Wire up the Gym Search
btn_gym = widgets.Button(
    description='Search the Gyms',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search the Gym',
    icon=''
)
display(btn_gym)

def btn_gym_clicked(b):
    print("Searching the Gyms...")
    response = conduct_search('gyms')

btn_gym.on_click(btn_gym_clicked)

# Wire up the supermarket search
btn_grocery = widgets.Button(
    description='Search the Supermarkets and Grocery',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search the Grocery',
    icon=''
)
display(btn_grocery)

def btn_grocery_clicked(b):
    print("Searching the supermarkets and Grocery stores...")
    response = conduct_search('grocerys')

btn_grocery.on_click(btn_grocery_clicked)

# Wire up the pub search
btn_pub = widgets.Button(
    description='Search the Pubs',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search the Pubs',
    icon=''
)
display(btn_pub)

def btn_pub_clicked(b):
    print("Searchings the Pubs...")
    response = conduct_search('pubs')

    
btn_pub.on_click(btn_pub_clicked)

### Now that you know WHO the human is, and WHERE the human is - you should have enough infromation to continue ###
for example:
If your target is in the gym, you know they will be interested in fitness topics.  Leverage that information with the bot named **Polly** and start a conversation with the human.

![title](img/stophere.jpg)

When the instructor indicates it is time to continue - click [here](Polly-100.ipynb) to go to the next lab